Домашнее задание #2.

SMS Spam/Ham dataset.
1(+6). Проверить, сбалансирован ли датасет (может быть, наблюдений одного класса слишком много?). Какие результаты покажет dummy classifier, который будет всем новым наблюдениям присваивать класс ham? Насколько плохо такое решение для задачи определения спама?
Грубое решение - включить в training set только необходимое число наблюдений (примерно поровну spam и ham). 
Нормализовать тексты и обучить байесовскую модель (bag of words). Проверить, как влияют на результат:
1) разная токенизация: в одном случае знаки препинания удалять, в другом — считать их токенами;
2) лемматизация (отсутствие лемматизации, стемминг, лемматизация; инструменты можно использовать любые, например, nltk.stem);
3) удаление стоп-слов, а также пороги минимальной и максимальной document frequency;
4) векторизация документов (CountVectorizer vs. TfIdfVectorizer);
5) что-нибудь ещё?
При оценке классификатора обратите внимание на TP и FP.

Extra: ограничив количество наблюдений ham в обучающей выборке, мы игнорируем довольно много данных. 1) В цикле: случайно выбрать нужное число писем ham и сконструировать сбалансированную выборку, построить классификатор, оценить и записать результат; в итоге результаты усреднить. 2) поможет ли параметр class prior probability?


2(+2). Сравнить результаты байесовского классификатора, решающего дерева и RandomForest. Помимо стандартных метрик оценки качества модели, необходимо построить learning curve, ROC-curve, classification report и интерпретировать эти результаты.

3(+2). А что, если в качестве предикторов брать не количество вхождений слов, а конструировать специальные признаки? Прежде всего, необходимо разделить таблицу на training set и test set в соотношении 80:20, test set не открывать до этапа оценки модели. С помощью pandas проверить, отличаются ли перечисленные ниже параметры (иможно придумать другие) для разных классов (spam/ham), и собрать матрицу признаков для обучения. Примеры признаков: длина сообщения, количество букв в ВЕРХНЕМ РЕГИСТРЕ, восклицательных знаков, цифр, запятых, каких-то конкретных слов (для этого можно построить частотный словарь по сообщениям каждого класса). Прокомментировать свой выбор. Векторизовать документы и построить классификатор. Оценить модель на проверочной выборке.


http://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
http://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html

In [2]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from nltk import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

messages = pandas.read_csv('./smsspamcollection/SMSSpamCollection',
                           sep='\t',
                           names=["label", "message"])
print(messages.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                 747
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                    4


In [3]:
#balansing the data
balansed_messages_ = messages.append(messages[messages.label == 'spam'])
balansed_messages = balansed_messages_.append(balansed_messages_[balansed_messages_.label == 'spam'])
print(balansed_messages.groupby('label').describe())

                                                        message
label                                                          
ham   count                                                4825
      unique                                               4516
      top                                Sorry, I'll call later
      freq                                                   30
spam  count                                                2988
      unique                                                653
      top     Please call our customer service representativ...
      freq                                                   16


In [4]:
#X, y = balansed_messages['message'], balansed_messages['label']
train, test = train_test_split(balansed_messages,
         test_size=0.2, random_state=42)
#print(train)

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
import re

st = LancasterStemmer()

lmtzr = WordNetLemmatizer()
tknzr = TweetTokenizer()
punct = re.compile(',|\.|;|:|\?|\)|\(|!')
non_letters = re.compile('[^a-zA-Z]')
stops = set(stopwords.words("english"))

def tokenize_np_nl_ds(text):
    #no punctuation, no lemmatisation, delete stops
    letters_only = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                 
    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    # Join the words back into one string separated by space, 
    # and return the result.
    return ( " ".join( meaningful_words ))

def tokenize_np_nl(text):
    #no punctuation, no lemmatisation, leave stops
    letters_only = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                   
    # Join the words back into one string separated by space, 
    # and return the result.
    return ( " ".join( words ))

def tokenize_p_nl_ds(text):
    #leave punct, no lemmatisation, delete stops
    txt = tknzr.tokenize(text)
    # Remove stop words
    meaningful_words = [w for w in txt if not w in stops]   
    # Join the words back into one string separated by space, 
    # and return the result.
    return ( " ".join( meaningful_words ))

def tokenize_p_nl(text):
    #leave punct, no lemmatisation, leave stops
    txt = tknzr.tokenize(text)
    # Join the words back into one string separated by space, 
    # and return the result.
    return ( " ".join( txt ))

def tokenize_p_l_ds(text):
    #leave punct, lemmetize, delete stops
    txt = tknzr.tokenize(text)
    txt = lmtzr.lemmatize(txt)
    meaningful_words = [w for w in txt if not w in stops]   
    return ( " ".join( meaningful_words ))

def tokenize_p_l(text):
    #leave punct, lemmetize, leave stops
    txt = tknzr.tokenize(text)
    txt = lmtzr.lemmatize(txt)  
    return ( " ".join( txt ))

def tokenize_np_l_ds(text):
    #delete punct, lemmetize, delete stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()
    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    txt = lmtzr.lemmatize(meaningful_words)
    return ( " ".join( txt ))

def tokenize_np_s(text):
    #delete punct, lemmetize, leave stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = txt.lower().split() 
    txt = st.stem(words)  
    return ( " ".join( txt ))

#same for stemmatisation
def tokenize_p_s_ds(text):
    #leave punct, lemmetize, delete stops
    txt = tknzr.tokenize(text)
    meaningful_words = [w for w in txt if not w in stops] 
    txt = st.stem(meaningful_words)
    return ( " ".join( txt ))

def tokenize_s_l(text):
    #leave punct, lemmetize, leave stops
    txt = tknzr.tokenize(text)
    txt = st.stem(txt)  
    return ( " ".join( txt ))

def tokenize_np_s_ds(text):
    #delete punct, lemmetize, delete stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = letters_only.lower().split()
    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]
    txt = st.stem(meaningful_words)
    return ( " ".join( txt ))

def tokenize_np_s(text):
    #delete punct, lemmetize, leave stops
    txt = non_letters.sub(" ", text) 
    # Convert to lower case, split into individual words
    words = txt.lower().split() 
    txt = st.stem(words)  
    return ( " ".join( txt ))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

parametrs = [tokenize_np_nl_ds,
              tokenize_np_nl,
              tokenize_p_nl_ds,
              tokenize_p_nl,
              #tokenize_p_l_ds,
              #tokenize_p_l,
              #tokenize_np_l_ds,
              #tokenize_np_s,
              #tokenize_p_s_ds,
              #tokenize_s_l,
              #tokenize_np_s_ds,
              #tokenize_np_s
            ]

clf = MultinomialNB()

for param in parametrs:
    cv = CountVectorizer(analyzer='word', tokenizer=param)
    cv.fit_transform(balansed_messages['message'])
    transformed_train = cv.transform(train['message'])
    transformed_test = cv.transform(test['message'])
    clf.fit(transformed_train, train['label'])
    print('For CountVectoriser and for '+ str(param))
    print('classification report is:')
    print(classification_report( clf.predict(transformed_test), test['label']))
    print()
    print('f1_score is:')
    #print(f1_score( clf.predict(transformed_test), test['label']))
    print()
    print('accuracy_score is:')
    print(accuracy_score( clf.predict(transformed_test), test['label']))
    print()
    print('confusion_matrix is:')
    print(confusion_matrix( clf.predict(transformed_test), test['label']))
    print()
    
    tcv = TfidfVectorizer(analyzer='word', tokenizer=param)
    tcv.fit_transform(balansed_messages['message'])
    transformed_train = tcv.transform(train['message'])
    transformed_test = tcv.transform(test['message'])
    clf.fit(transformed_train, train['label'])
    print('For TfIdfVectoriser and for '+ str(param))
    print('classification report is:')
    print(classification_report( clf.predict(transformed_test), test['label']))
    print()
    print('f1_score is:')
    #print(f1_score( clf.predict(transformed_test), test['label']))
    print()
    print('accuracy_score is:')
    print(accuracy_score( clf.predict(transformed_test), test['label']))
    print()
    print('confusion_matrix is:')
    print(confusion_matrix( clf.predict(transformed_test), test['label']))
    print()


For CountVectoriser and for <function tokenize_np_nl_ds at 0x114dbef28>
classification report is:
             precision    recall  f1-score   support

        ham       0.87      0.88      0.87       962
       spam       0.80      0.78      0.79       601

avg / total       0.84      0.84      0.84      1563


f1_score is:

accuracy_score is:
0.840690978887

confusion_matrix is:
[[843 119]
 [130 471]]

For TfIdfVectoriser and for <function tokenize_np_nl_ds at 0x114dbef28>
classification report is:
             precision    recall  f1-score   support

        ham       0.99      0.63      0.77      1525
       spam       0.05      0.84      0.10        38

avg / total       0.97      0.64      0.76      1563


f1_score is:

accuracy_score is:
0.63915547025

confusion_matrix is:
[[967 558]
 [  6  32]]

For CountVectoriser and for <function tokenize_np_nl at 0x11a1cfea0>
classification report is:
             precision    recall  f1-score   support

        ham       0.87      0.89    

In [7]:
from sklearn.model_selection import learning_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

#chose best vectosiser
cv = CountVectorizer(analyzer='word', tokenizer=tokenize_p_nl_ds)
cv.fit_transform(balansed_messages['message'])
transformed_train = cv.transform(train['message'])
transformed_test = cv.transform(test['message'])

#train DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(transformed_train, train['label'])
print(classification_report( dtc.predict(transformed_test), test['label']))

#train RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(transformed_train, train['label'])
print(classification_report( rfc.predict(transformed_test), test['label']))


             precision    recall  f1-score   support

        ham       0.98      1.00      0.99       951
       spam       1.00      0.96      0.98       612

avg / total       0.99      0.99      0.99      1563

             precision    recall  f1-score   support

        ham       0.99      1.00      0.99       966
       spam       0.99      0.98      0.99       597

avg / total       0.99      0.99      0.99      1563



In [19]:
print(train['message'].size)
print(test['message'].size)

plt.figure()
plt.title('smth')

plt.xlabel("Training examples")
plt.ylabel("Score")
train_sizes, train_scores, test_scores = learning_curve(
            dtc, transformed_train, train['label'], train_sizes=(range(1, 4167)))
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.grid()

#plt.fill_between(train['message'].size, train_scores_mean - train_scores_std,
#                 train_scores_mean + train_scores_std, alpha=0.1, color="r")

#plt.fill_between(train['message'].size, test_scores_mean - test_scores_std,
#                 test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(np.linspace(.1, 1.0, 5), train_scores_mean, 'o-', color="r",
         label="Training score")
#plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
#         label="Cross-validation score")

#plt.legend(loc="best")
plt.show()

6250
1563


ValueError: x and y must have same first dimension, but have shapes (5,) and (4166,)